In [ ]:
"""
Purpose: To decompose the multi-somas for splitting
using the new decomposition method




"""


In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")
sys.path.append("/meshAfterParty/meshAfterParty")



In [2]:
import datajoint_utils as du
from importlib import reload

INFO - 2021-04-20 02:11:45,958 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-20 02:11:45,960 - settings - Setting database.user to celiib
INFO - 2021-04-20 02:11:45,961 - settings - Setting database.password to newceliipass
INFO - 2021-04-20 02:11:45,963 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2021-04-20 02:11:45,964 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-20 02:11:45,975 - connection - Connect

Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2021-04-20 02:11:46,376 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-20 02:11:46,379 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-20 02:11:46,380 - settings - Setting database.user to celiib
INFO - 2021-04-20 02:11:46,381 - settings - Setting database.password to newceliipass
INFO - 2021-04-20 02:11:46,382 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-20 02:11:46,383 - settings - Setting database.user to celiib
INFO - 2021-04-20 02:11:46,383 - settings - Setting database.password to newceliipass
INFO - 2021-04-20 02:11:46,386 - settings - Setting enable_python_native_blobs to True
INFO - 2021-04-20 02:11:46,830 - settings - Setting enable_python_native_blobs to True


In [3]:
#so that it will have the adapter defined
from datajoint_utils import *

In [4]:
test_mode = True

# Debugging the contains method

In [5]:
import system_utils as su

In [6]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 200)
print(f"Sleeping {random_sleep_sec} sec before conneting")
if not test_mode:
    time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

INFO - 2021-04-20 02:11:46,971 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-20 02:11:46,972 - settings - Setting database.user to celiib
INFO - 2021-04-20 02:11:46,973 - settings - Setting database.password to newceliipass
INFO - 2021-04-20 02:11:46,975 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2021-04-20 02:11:46,976 - settings - Setting database.user to celiib
INFO - 2021-04-20 02:11:46,976 - settings - Setting database.password to newceliipass
INFO - 2021-04-20 02:11:46,982 - settings - Setting enable_python_native_blobs to True


Sleeping 77 sec before conneting
Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


INFO - 2021-04-20 02:11:47,472 - settings - Setting enable_python_native_blobs to True


In [7]:
from importlib import reload

import neuron_utils as nru

import neuron

import neuron_visualizations as nviz

import time

import datajoint_utils as du

import numpy as np

import proofreading_utils as pru

import preprocessing_vp2 as pre

# -- For the axon classification --

import neuron_searching as ns

import skeleton_utils as sk

import numpy_utils as nu

import networkx_utils as xu

import system_utils as su

import classification_utils as clu
import proofreading_utils as pru

import datajoint as dj

from pykdtree.kdtree import KDTree
import trimesh_utils as tu
import proofreading_utils as pru
import numpy as np

# Defining the Table

In [8]:
import neuron_utils as nru
import neuron
import trimesh_utils as tu
import numpy as np

In [9]:
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 8043


# Defining the Synapse Table

In [10]:
@schema
class AutoProofreadValidationSynapse4(dj.Manual):
    definition="""
    synapse_id           : bigint unsigned              # synapse index within the segmentation
    synapse_type: enum('presyn','postsyn')
    ver                  : decimal(6,2)                 # the version number of the materializaiton
    ---
    segment_id           : bigint unsigned              # segment_id of the cell. Equivalent to Allen 'pt_root_id
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    nucleus_id           : int unsigned                 # id of nucleus from the flat segmentation  Equivalent to Allen: 'id'. 
    skeletal_distance_to_soma=NULL : double #the length (in um) of skeleton distance from synapse to soma (-1 if on the soma)
    """

# Defining the Proofreading Stats Table

In [11]:
"""
This table will include the following information:

1) Filtering Info
2) Synapse Stats for Individual Neuron
3) Synapse Stats for Segment


**** thing need to add:
1) Axon faces
2) Axon length/area
2) Neuron faces
3) n_presyn_error_syn_non_axon
"""

'\nThis table will include the following information:\n\n1) Filtering Info\n2) Synapse Stats for Individual Neuron\n3) Synapse Stats for Segment\n\n\n**** thing need to add:\n1) Axon faces\n2) Axon length/area\n2) Neuron faces\n3) n_presyn_error_syn_non_axon\n'

In [12]:
@schema
class AutoProofreadValidationStats4(dj.Manual):
    definition="""
    -> minnie.Decomposition()
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    proof_version    : tinyint unsigned             # the version of code used for this cell typing classification
    ---
    mesh_faces: <faces>                      # faces indices that were saved off as belonging to proofread neuron (external storage)
    axon_faces: <faces>                      # faces indices that were saved off as belonging to proofread neuron's axon (external storage)
    
    axon_on_dendrite_merges_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    axon_on_dendrite_merges_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    low_branch_clusters_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    low_branch_clusters_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    dendrite_on_axon_merges_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    dendrite_on_axon_merges_error_length =NULL: double #the length (in um) of skeleton distance canceled out by filter
    
    double_back_and_width_change_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    double_back_and_width_change_error_length =NULL: double #the length (in um) of skeleton distance canceled out by filter
    
    crossovers_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    crossovers_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    high_degree_coordinates_error_area=NULL : double #the area (in um ^ 2) of the faces canceled out by filter
    high_degree_coordinates_error_length=NULL : double #the length (in um) of skeleton distance canceled out by filter
    
    # ------------ For local valid synapses to that split_index
    n_valid_syn_presyn_for_split: int unsigned
    n_valid_syn_postsyn_for_split : int unsigned
    n_presyn_error_syn_non_axon :int unsigned
    presyn_error_syn_non_axon_ids=NULL : longblob #the ids of the synapses that were errored out
    
    # ------------ For global stats belonging to the whole segment
    # For the whole segment
    n_presyn_error_syn: int unsigned
    n_postsyn_error_syn: int unsigned
    total_error_synapses: int unsigned
    
    total_presyns: int unsigned 
    total_postsyns: int unsigned 
    total_synapses:int unsigned
    
    perc_error_presyn=NULL: double
    perc_error_postsyn=NULL: double
    
    overall_percent_error=NULL: double
    """
    
    

# Creating the Auto Proofread Neuron Table

In [13]:
# minnie,schema = du.configure_minnie_vm()
# minnie.AutoProofreadValidationNeurons3.drop()
# minnie.AutoProofreadValidationStats3.drop()
# minnie.AutoProofreadValidationSynapse3.drop()
# minnie.schema.external['faces'].delete(delete_external_files=True)

In [14]:
key_source = (minnie.Decomposition() 
                  & (minnie.AutoProofreadValidationSegment() - minnie.AutoProofreadValidationSegmentExclude() ))
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses),ver the version number of the materializaiton,process_version the version of the preprocessing pipeline run,index the index of the neuron object that resulted from that mesh (indexed starting at 0),multiplicity the number of somas found for this base segment,decomposition,n_vertices number of vertices,n_faces number of faces,n_not_processed_soma_containing_meshes the number of meshes with somas that were not processed,n_error_limbs the number of limbs that are touching multiple somas or 1 soma in multiple places,n_same_soma_multi_touching_limbs number of limbs that touch the same soma multiple times,n_multi_soma_touching_limbs number of limbs that touch multiple somas,n_somas number of soma meshes detected,n_limbs,n_branches,max_limb_n_branches,skeletal_length,max_limb_skeletal_length,median_branch_length gives information on average skeletal length to next branch point,width_median median width from mesh center without spines removed,width_no_spine_median median width from mesh center with spines removed,width_90_perc 90th percentile for width without spines removed,width_no_spine_90_perc 90th percentile for width with spines removed,n_spines,spine_density n_spines/ skeletal_length,spines_per_branch,skeletal_length_eligible the skeletal length for all branches searched for spines,n_spine_eligible_branches the number of branches that were checked for spines because passed width threshold,spine_density_eligible n_spines/skeletal_length_eligible,spines_per_branch_eligible n_spines/n_spine_eligible_branches,total_spine_volume the sum of all spine volume,spine_volume_median median of the spine volume for those spines with able to calculate volume,spine_volume_density total_spine_volume/skeletal_length,spine_volume_density_eligible total_spine_volume/skeletal_length_eligible,spine_volume_per_branch_eligible total_spine_volume/n_spine_eligible_branches,run_time the amount of time to run (seconds)
864691135113167769,0,0.25,30.00,5,0,1,=BLOB=,452717,923821,0,0,0,0,1,8,151,71,5692444.08267742,2394232.4798491327,24167.929,110.163,110.163,417.822,371.702,1335,0.0002345214077837876,8.841059602649006,3385028.367,70,0.0003943836964601721,19.071,254408283170.89862,122787553.96568298,44692.276195577244,75156.91320376418,3634404045.2985516,1253.1528
864691135212632448,0,0.25,30.00,5,0,1,=BLOB=,627122,1290246,0,1,1,0,1,6,185,70,7318104.475983632,2277192.467242251,30429.29,95.346,95.346,433.323,421.42,1921,0.00026249966863745776,10.383783783783784,4385015.158,81,0.0004380828642052325,23.716,413279413191.4511,139194450.5361525,56473.56013401297,94248.11506921845,5102214977.6722355,2066.996
864691135212690816,0,0.25,30.00,5,0,1,=BLOB=,354542,734849,0,0,0,0,1,8,68,27,3497842.1349936416,1089750.912749487,41748.835,283.913,257.061,590.615,562.343,1192,0.0003407815315833763,17.529411764705884,2851734.625,46,0.00041799120771975756,25.913,191408781914.96283,106143594.09584618,54721.961291518026,67120.1240946334,4161060476.4122353,2014.3769
864691135233108569,0,0.25,30.00,5,0,1,=BLOB=,175149,369334,0,0,0,0,1,4,51,26,1929287.5881296252,674706.4840692146,24642.666,125.806,125.806,408.318,362.169,485,0.00025138813051204563,9.509803921568627,1349577.796,27,0.00035937165048023655,17.963,151132346098.62347,251855833.4273572,78335.83081573693,111984.90857404671,5597494299.949018,525.6969
864691135272164113,0,0.25,30.00,5,0,1,=BLOB=,728741,1472234,0,0,0,0,1,9,120,28,7083748.5002949275,1812689.1716514435,46773.703,325.423,284.002,518.067,518.067,2746,0.0003876478674935554,22.883333333333333,5709127.65,89,0.00048098416576830257,30.854,491376863834.10925,117347405.39998308,69366.78565220816,86068.64199894168,5521088357.686621,4975.5419
864691135348272855,0,0.25,30.00,5,0,1,=BLOB=,881685,1783251,0,0,0,0,1,9,191,74,9439614.600445235,2621310.0931448657,33916.923,134.159,134.15

In [ ]:
import numpy as np
import time
import classification_utils as clu

proof_version = 1
proof_version = 2 
proof_version = 3
proof_version = 4

verbose = True
ver = 0.08

@schema
class AutoProofreadValidationNeurons3(dj.Computed):
    definition="""
    -> minnie.Decomposition()
    split_index          : tinyint unsigned             # the index of the neuron object that resulted AFTER THE SPLITTING ALGORITHM
    proof_version    : tinyint unsigned             # the version of code used for this cell typing classification
    ---
    multiplicity  : tinyint unsigned   # the total number of neurons that came from the parent segment id
    # -------- Important Excitatory Inhibitory Classfication ------- #
    cell_type_predicted: enum('excitatory','inhibitory','other','unknown') # morphology predicted by classifier
    spine_category: enum('no_spined','sparsely_spined','densely_spined')
    
    n_axons: tinyint unsigned             # Number of axon candidates identified
    n_apicals: tinyint unsigned             # Number of apicals identified
    
    axon_length: double  # length (in um) of the classified axon skeleton
    axon_area: double # # area (in um^2) of the classified axon
    
    # ----- Soma Information ----#
    nucleus_id           : int unsigned                 # id of nucleus from the flat segmentation  Equivalent to Allen: 'id'.
    nuclei_distance      : double                    # the distance to the closest nuclei (even if no matching nuclei found)
    n_nuclei_in_radius   : tinyint unsigned          # the number of nuclei within the search radius of 15000 belonging to that segment
    n_nuclei_in_bbox     : tinyint unsigned          # the number of nuclei within the bounding box of that soma
    
    soma_x            : int unsigned                 # x coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)
    soma_y            : int unsigned                 # y coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)
    soma_z            : int unsigned                 # z coordinate of nucleus centroid in EM voxels (x: 4nm, y: 4nm, z: 40nm)
    
    max_soma_n_faces     : int unsigned                 # The largest number of faces of the somas
    max_soma_volume      : int unsigned                 # The largest volume of the somas the (volume in billions (10*9 nm^3))
    
    # ---- Stores Neuron Mesh Faces (moved to AutoProofreadStats) --------
    
    
    # ------------- The Regular Neuron Information ----------------- #
    n_vertices           : int unsigned                 # number of vertices
    n_faces              : int unsigned                 # number of faces
    n_not_processed_soma_containing_meshes : int unsigned  #the number of meshes with somas that were not processed
    n_error_limbs: int #the number of limbs that are touching multiple somas or 1 soma in multiple places
    n_same_soma_multi_touching_limbs: int # number of limbs that touch the same soma multiple times
    n_multi_soma_touching_limbs: int # number of limbs that touch multiple somas
    n_somas: int #number of soma meshes detected
    n_limbs: int
    n_branches: int
    max_limb_n_branches=NULL:int
    
    skeletal_length=NULL: double
    max_limb_skeletal_length=NULL:double
    median_branch_length=NULL:double #gives information on average skeletal length to next branch point
    
    
    width_median=NULL: double #median width from mesh center without spines removed
    width_no_spine_median=NULL: double #median width from mesh center with spines removed
    width_90_perc=NULL: double # 90th percentile for width without spines removed
    width_no_spine_90_perc=NULL: double  # 90th percentile for width with spines removed
    
    
    n_spines: bigint

    spine_density=NULL: double # n_spines/ skeletal_length
    spines_per_branch=NULL: double
    
    skeletal_length_eligible=NULL: double # the skeletal length for all branches searched for spines
    n_spine_eligible_branches=NULL: int # the number of branches that were checked for spines because passed width threshold
    
    spine_density_eligible=NULL:double # n_spines/skeletal_length_eligible
    spines_per_branch_eligible=NULL:double # n_spines/n_spine_eligible_branches
    
    total_spine_volume=NULL: double # the sum of all spine volume
    spine_volume_median=NULL: double # median of the spine volume for those spines with able to calculate volume
    spine_volume_density=NULL: double #total_spine_volume/skeletal_length
    spine_volume_density_eligible=NULL: double #total_spine_volume/skeletal_length_eligible
    spine_volume_per_branch_eligible=NULL: double #total_spine_volume/n_spine_eligible_branches
    
    
    
    
    # ------ Information Used For Excitatory Inhibitory Classification -------- 
    axon_angle_maximum=NULL:double #the anlge of an identified axon
    spine_density_classifier:double              # the number of spines divided by skeletal length for branches analyzed in classification
    n_branches_processed: int unsigned                 # the number branches used for the spine density analysis
    skeletal_length_processed: double                 # The total skeletal length of the viable branches used for the spine density analysis
    n_branches_in_search_radius: int unsigned                 # the number branches existing in the search radius used for spine density
    skeletal_length_in_search_radius : double         # The total skeletal length of the branches existing in the search radius used for spine density
    
    

    run_time=NULL : double                   # the amount of time to run (seconds)

    
    """
                             
        
    
        
        
    
    
    
    key_source = (minnie.Decomposition() 
                  & (minnie.AutoProofreadValidationSegment() - minnie.AutoProofreadValidationSegmentExclude() )
                 )
    
    

    def make(self,key):
        """
        Pseudocode:
        1) Pull Down All of the Neurons
        2) Get the nucleus centers and the original mesh

        """
        
        # 1) Pull Down All of the Neurons
        segment_id = key["segment_id"]
        print(f"key = {key}")
        
        
        whole_pass_time = time.time()
        
#         if ver is None:
#             ver = key["ver"]

        curr_output = pru.proofreading_table_processing(key,
                                  proof_version=proof_version,
                                  compute_synapse_to_soma_skeletal_distance=True,
                                    return_errored_synapses_ids_non_axons=True,
                                                        ver=ver,
                                                        validation=True,
                                 verbose=True,)    
        # ------ Writing the Data To the Tables ----- #
            
            
        AutoProofreadSynapse_keys = curr_output["AutoProofreadSynapse_keys"]
        AutoProofreadNeurons_keys = curr_output["AutoProofreadNeurons_keys"]
        filtering_info_list = curr_output["filtering_info_list"]
        synapse_stats_list = curr_output["synapse_stats_list"]
        total_error_synapse_ids_list = curr_output["total_error_synapse_ids_list"]
        neuron_mesh_list = curr_output["neuron_mesh_list"]
        axon_mesh_list = curr_output["axon_mesh_list"]
        neuron_split_idxs = curr_output["neuron_split_idxs"]
            
        
        # Once have inserted all the new neurons need to compute the stats
        if verbose:
            print("Computing the overall stats")
            
        overall_syn_error_rates = pru.calculate_error_rate(total_error_synapse_ids_list,
                        synapse_stats_list,
                        verbose=True)
        
        
        # Final Part: Create the stats table entries and insert
        
        proofread_stats_entries = []
        
        stats_to_make_sure_in_proofread_stats = [
            
         'axon_on_dendrite_merges_error_area',
         'axon_on_dendrite_merges_error_length',
         'low_branch_clusters_error_area',
         'low_branch_clusters_error_length',
         'dendrite_on_axon_merges_error_area',
         'dendrite_on_axon_merges_error_length',
         'double_back_and_width_change_error_area',
         'double_back_and_width_change_error_length',
         'crossovers_error_area',
         'crossovers_error_length',
         'high_degree_coordinates_error_area',
         'high_degree_coordinates_error_length',
        ]
        
        
        for sp_idx,split_index in enumerate(neuron_split_idxs):
            
            #write the AutoProofreadNeurons and AutoProofreadSynapse Tabel
            keys_to_write = AutoProofreadSynapse_keys[sp_idx]
            AutoProofreadValidationSynapse3.insert(keys_to_write,skip_duplicates=True)
            
            new_key = AutoProofreadNeurons_keys[sp_idx]
            self.insert1(new_key,skip_duplicates=True,allow_direct_insert=True)
            
            
            
            
            
            synapse_stats = synapse_stats_list[sp_idx]
            filtering_info = filtering_info_list[sp_idx]
            
            curr_key = dict(key,
                           split_index = split_index,
                           proof_version = proof_version,
                           
                            mesh_faces = neuron_mesh_list[sp_idx],
                            axon_faces = axon_mesh_list[sp_idx],
                         

                            # ------------ For local valid synapses to that split_index
                            n_valid_syn_presyn_for_split=synapse_stats["n_valid_syn_presyn"],
                            n_valid_syn_postsyn_for_split=synapse_stats["n_valid_syn_postsyn"],
                            n_presyn_error_syn_non_axon=synapse_stats["n_errored_syn_presyn_non_axon"],
                            presyn_error_syn_non_axon_ids = synapse_stats["presyn_error_syn_non_axon_ids"]
                           
                           
                           )
            
            
            for s in stats_to_make_sure_in_proofread_stats:
                if s not in filtering_info.keys():
                    curr_key[s] = None
            
            filter_key = {k:np.round(v,2) for k,v in filtering_info.items() if "area" in k or "length" in k}
            curr_key.update(filter_key)
            curr_key.update(overall_syn_error_rates)
            
            proofread_stats_entries.append(curr_key)
            
        
        AutoProofreadValidationStats3.insert(proofread_stats_entries,skip_duplicates=True)

            

        print(f"\n\n ***------ Total time for {key['segment_id']} = {time.time() - whole_pass_time} ------ ***")
    

# Running the Populate

In [ ]:
curr_table = (minnie.schema.jobs & "table_name='__auto_proofread_validation_neurons3'")
((curr_table) & "timestamp>'2021-02-22'")#.delete()
#curr_table.delete()
#(curr_table & "error_message = 'ValueError: need at least one array to concatenate'").delete()

In [ ]:
#minnie.Decomposition() & minnie.AutoProofreadValidationSegment() - minnie.AutoProofreadValidationSegmentExclude()

In [ ]:
import time
pru = reload(pru)
nru = reload(nru)
import neuron_searching as ns
ns = reload(ns)
clu = reload(clu)
import random

start_time = time.time()
if not test_mode:
    time.sleep(random.randint(0, 800))
print('Populate Started')
if not test_mode:
    AutoProofreadValidationNeurons3.populate(reserve_jobs=True, suppress_errors=True, order="random")
else:
    AutoProofreadValidationNeurons3.populate(reserve_jobs=True, suppress_errors=False, order="random")
print('Populate Done')

print(f"Total time for AutoProofreadValidationNeurons3 populate = {time.time() - start_time}")